In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2
from IPython.display import HTML, display
import matplotlib.pyplot as plt
import mediapipe as mp


2025-05-14 01:12:15.492973: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747185135.519414    3591 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747185135.525573    3591 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747185135.550781    3591 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747185135.550807    3591 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747185135.550809    3591 computation_placer.cc:177] computation placer alr

In [9]:
#눈 감았는지 계산 함수
def calculate_EAR(landmarks, eye_indices, image_w, image_h):
    points = [(int(landmarks[i].x * image_w), int(landmarks[i].y * image_h)) for i in eye_indices]
    A = np.linalg.norm(np.array(points[1]) - np.array(points[5]))
    B = np.linalg.norm(np.array(points[2]) - np.array(points[4]))
    C = np.linalg.norm(np.array(points[0]) - np.array(points[3]))
    ear = (A + B) / (2.0 * C)
    return ear

In [10]:
# 왼쪽, 오른쪽 눈 인덱스
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]

In [11]:
def calculate_MAR(landmarks, image_w, image_h):
    top = np.array([landmarks[13].x * image_w, landmarks[13].y * image_h])
    bottom = np.array([landmarks[14].x * image_w, landmarks[14].y * image_h])
    left = np.array([landmarks[78].x * image_w, landmarks[78].y * image_h])
    right = np.array([landmarks[308].x * image_w, landmarks[308].y * image_h])
    
    vertical = np.linalg.norm(top - bottom)
    horizontal = np.linalg.norm(left - right)
    
    return vertical / horizontal


In [12]:
# 표정 판단
def classify_expression(ear, mar):
    if ear < 0.20 and mar < 0.3:
        return "차분함"
    elif ear > 0.23 or mar > 0.35:
        return "흥미로움"
    else:
        return "차분함"

In [18]:
# 이미지 불러오기
path = "/workspaces/Focussu-AI/data/WIN_20250512_22_01_55_Pro.jpg"
image = cv2.imread(path)
rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [14]:
mp_face_mesh = mp.solutions.face_mesh


In [19]:
with mp_face_mesh.FaceMesh(static_image_mode=True, refine_landmarks=True) as face_mesh:
    results = face_mesh.process(rgb)
    h, w, _ = image.shape

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmarks = face_landmarks.landmark

            ear_left = calculate_EAR(landmarks, LEFT_EYE, w, h)
            ear_right = calculate_EAR(landmarks, RIGHT_EYE, w, h)
            ear_avg = (ear_left + ear_right) / 2.0

            mar = calculate_MAR(landmarks, w, h)

            expression = classify_expression(ear_avg, mar)
            print("감정 인식 결과:", expression)
    else:
        print("얼굴을 찾을 수 없습니다.")

감정 인식 결과: 차분함


W0000 00:00:1747186123.491426    5943 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747186123.519627    5942 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [ ]:
# 눈 떴는지 판단단
with mp_face_mesh.FaceMesh(static_image_mode=True, refine_landmarks=True) as face_mesh:
    results = face_mesh.process(rgb)
    h, w, _ = image.shape

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmarks = face_landmarks.landmark
            
            left_EAR = calculate_EAR(landmarks, LEFT_EYE, w, h)
            right_EAR = calculate_EAR(landmarks, RIGHT_EYE, w, h)

            avg_EAR = (left_EAR + right_EAR) / 2.0
            print(f"EAR: {avg_EAR:.3f}")

            if avg_EAR < 0.20:
                print("🔒 눈 감았음!")
            else:
                print("👁️ 눈 떠 있음")

EAR: 0.332
👁️ 눈 떠 있음


W0000 00:00:1747185161.539927    3738 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747185161.558936    3738 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [ ]:
# 얼굴 랜드마크 추출 
with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,       # 눈, 입술 디테일 향상
    min_detection_confidence=0.5
) as face_mesh:
    results = face_mesh.process(rgb)
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            for idx, landmark in enumerate(face_landmarks.landmark):
                h, w, _ = image.shape
                x = int(landmark.x * w)
                y = int(landmark.y * h)
                cv2.circle(image, (x, y), 1, (0, 255, 0), -1)
    else:
        print("얼굴을 감지하지 못했습니다.")



W0000 00:00:1747184804.745790    2779 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747184804.791982    2778 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [ ]:
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(8, 8))
plt.imshow(image_rgb)
plt.axis("off")
plt.title("Face Landmarks")
plt.show()

In [ ]:
left_iris = [468, 469, 470, 471, 472]
right_iris = [473, 474, 475, 476, 477]

# 눈, 동공만 랜드마크 추출 
with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,       # 눈, 입술 디테일 향상
    min_detection_confidence=0.5
) as face_mesh:
    results = face_mesh.process(rgb)


    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            h, w, _ = image.shape
            for idx in left_iris + right_iris:
                landmark = face_landmarks.landmark[idx]
                x = int(landmark.x * w)
                y = int(landmark.y * h)
                cv2.circle(image, (x, y), 2, (0, 0, 255), -1)  # 빨간 점
    else:
        print("얼굴을 감지하지 못했습니다.")



W0000 00:00:1747184831.458745    2868 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747184831.496243    2867 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [ ]:
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(8, 8))
plt.imshow(image_rgb)
plt.axis("off")
plt.title("Face Landmarks")
plt.show()